In [ ]:

!pip install spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os
import time
import numpy as np
import re
from tqdm import tqdm
from configparser import ConfigParser
 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sleep_min = 1
sleep_max = 3
configur = ConfigParser()
configur.read('config.ini')
client_id = '3be03b1aa6284c118c88fc57d4195465'
client_secret = '3290e9d2833a4ffdba883020763ac941'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # sp is the instance of the spotipy api

In [ ]:
data = pd.read_csv('LY_Artist_lyrics_genre_data_from_big5_mft_users_likes_final.csv')
data = data[data['lang_detect_spacy']=='en']
sample_dataset = data[:1000]

In [ ]:
artists = list(sample_dataset['Artist'])
titles = list(sample_dataset['title'])

</h2> Data Cleaning


In [ ]:
def pre_process_text(text):
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text


In [ ]:
'''
check how many artists and titles have special characters
'''
c = 0
for title in titles:
    if str(title).isascii():
        c+=1
print('The number of non ascii characters in our sample dataset titles are:- ', 200-c)

The number of non ascii characters in our sample dataset titles are:-  -698


In [ ]:
artists_clean = []
titles_clean = []

for i in tqdm(range(len(artists))):
    artists_clean.append(pre_process_text(artists[i]))
    titles_clean.append(pre_process_text(titles[i]))

100%|██████████| 1000/1000 [00:00<00:00, 99553.87it/s]


In [ ]:
request_count = 0
start_time = time.time()
spotify_artists = []
check_dat = []
for i in tqdm(range(len(artists))):
    query = sp.search(f'artist:{artists_clean[i]} track:{titles_clean[i]}')
    if query['tracks']['items']:
        spotify_artists.append(query['tracks']['items'][0]['artists'][0]['name'])
        check_dat.append(query['tracks']['items'][0]['artists'][0]['name'])
    else:
        spotify_artists.append(artists[i])
        
    request_count+=1
    if request_count % 5 == 0:
        # print(str(request_count) + " requests sent")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        # print('Elapsed Time: {} seconds'.format(time.time() - start_time))
print('Completed...')

100%|██████████| 1000/1000 [08:43<00:00,  1.91it/s]

Completed...


In [ ]:
print(f'Out of the 1000 sample data records, data for {len(check_dat)} records is available')

Out of the 1000 sample data records, data for 911 records is available


In [ ]:
df1 = pd.DataFrame(spotify_artists)
df2 = pd.DataFrame(artists)
df3 = pd.concat([df2,df1], axis=1, join='outer')
df3.to_csv('check.csv')

Checking if the artisrts that we extracted from spotify are same as in our dataset

In [ ]:
count = 0
for i in tqdm(range(len(artists))):
    if pre_process_text(artists[i]) in pre_process_text(spotify_artists[i]):
        count+=1
count

100%|██████████| 1000/1000 [00:00<00:00, 102753.72it/s]


1000

Extracting Data

In [ ]:
'''
In track_preview_available function I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available.

In is_same_artist function I check whether the artist the we found in our query is the same one from our dataset.
'''
def is_same_artist(query, data, index):
    if pre_process_text(data[index]) in pre_process_text(query['tracks']['items'][0]['artists'][0]['name']):
        return True

def track_preview_available(query):
    if query['tracks']['items']:
        if query['tracks']['items'][0]['preview_url']:
            return True
        return True
        
def artist_available(query):
    if query['tracks']['items']:
        return True

def is_same_track(query, data , index):
    x['tracks']['items'][0]['name'] # track name

In [ ]:
query = sp.search('coldplay fix you', type='track')
query['tracks']['items'][0]

In [ ]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()

preview_url = []
track_id = []
artist_id = []
track_popularity = []
artists_df = []
titles_df = []

for index in range(len(spotify_artists)):
    artist_name = spotify_artists[index]
    song_title = titles_clean[index]
    search = f'artist:{artist_name} track:{song_title}'
    query = sp.search(search, type='track')


    # in the below line of code I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available, and
    # even further I check a third condition whether the artist the we found in our query is the same from our dataset.


    # if query['tracks']['items'] and query['tracks']['items'][0]['preview_url'] and query['tracks']['items'][0]['artists'][0]['name'] in alpha_artists_titles[index]:
    if track_preview_available(query) and is_same_artist(query,artists,index):
        preview_url.append(query['tracks']['items'][0]['preview_url'])
        track_id.append(query['tracks']['items'][0]['id'])
        artist_id.append(query['tracks']['items'][0]['artists'][0]['id'])
        track_popularity.append(query['tracks']['items'][0]['popularity'])
        artists_df.append(spotify_artists[index])
        titles_df.append(titles_clean[index])
         
    else:
        preview_url.append(None)
        track_id.append(None)
        artist_id.append(None)
        track_popularity.append(None)
        artists_df.append(spotify_artists[index])
        titles_df.append(titles_clean[index])


    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

In [ ]:
track_data = pd.DataFrame(artists_df, columns=['artists'])
track_data['tracks'] = titles_df
track_data['artist_id'] = artist_id
track_data['track_id'] = track_id
track_data['track_popularity'] = track_popularity
track_data['track_url'] = preview_url

In [ ]:
track_data

,artists,tracks,artist_id,track_id,track_popularity,track_url
0,Alesana,curse of the virgin canvas,1fnDlUNi67FQtHHqBl4Yfg,6q2RsgScL1R3dG9pAbZzhy,43.0,None
1,Alesana,the artist,1fnDlUNi67FQtHHqBl4Yfg,1SCJh0eUMRLsm9xKEb6W2z,39.0,None
2,Alesana,"congratulations, i hate you",1fnDlUNi67FQtHHqBl4Yfg,2wiRaJG78MwRqBWnDSoO4H,40.0,https://p.scdn.co/mp3-preview/d657219662766b9b...
3,Alesana,a lunatic s lament,1fnDlUNi67FQtHHqBl4Yfg,3soK40VuGFJaqY5BeHG5c6,39.0,None
4,Alesana,ambrosia,1fnDlUNi67FQtHHqBl4Yfg,0CsngWR4MiYHiDbBebVxql,45.0,None
...,...,...,...,...,...,...
995,Anonymous,is getting rich worth it?,None,None,NaN,None
996,Anonymous,the grave,4kCZ5nyurc9eIqLJfUcW0Y,7JuxR74u5BSZpfIvNdRvFb,0.0,https://p.scdn.co/mp3-preview/c053a3ed0ce07b7e...
997,Anonymous,alisoun,None,None,NaN,None
998,Anouk,it s a new day,6ltVunYjAAD70YtVO6rxvX,1nhXX3flwQf9Rq4xL9ixJL,37.0,None


## Audio Features

In [ ]:
def get_audio_features(track_id, chunk_size):
    audio_features_list = []

    for i in range(0, len(track_id), chunk_size):    
        track_id_list = track_id[i:i+chunk_size]
        results = sp.audio_features(track_id_list)
        results = [v for v in results if str(v) != 'None']
        audio_features_list.extend(results)

    return audio_features_list

In [ ]:
# remove NaN and duplicate track id's
ids = [x for x in track_id if str(x) != 'None']
ids = list(dict.fromkeys(ids))
ids

In [ ]:
x = get_audio_features(ids, 100)
x

In [ ]:
features_df = pd.DataFrame(x, columns = ['id', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'tempo', 'mode', 'speechiness', 'valence'])
features_df = features_df.rename(columns = {'id':'track_id'})
features_df

,track_id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,tempo,mode,speechiness,valence
0,6q2RsgScL1R3dG9pAbZzhy,0.009310,0.228,0.8780,0.000114,2,0.750,-5.232,193.133,0,0.1290,0.161
1,1SCJh0eUMRLsm9xKEb6W2z,0.003200,0.264,0.9790,0.000002,2,0.228,-3.378,174.119,0,0.2390,0.472
2,2wiRaJG78MwRqBWnDSoO4H,0.000058,0.318,0.9190,0.011800,2,0.104,-5.928,133.762,0,0.0879,0.186
3,3soK40VuGFJaqY5BeHG5c6,0.000794,0.315,0.9160,0.001690,2,0.657,-4.647,186.084,0,0.0892,0.136
4,0CsngWR4MiYHiDbBebVxql,0.000038,0.275,0.9840,0.482000,7,0.112,-4.328,119.376,0,0.0601,0.319
...,...,...,...,...,...,...,...,...,...,...,...,...
899,6vSY2O8TWgHP0oLPn5WhbH,0.010700,0.598,0.7600,0.000011,2,0.343,-5.822,120.488,0,0.0295,0.164
900,3v7tXPVFdJxa1BAxp4MjiE,0.994000,0.337,0.0317,0.058900,2,0.103,-20.302,64.894,0,0.0490,0.201
901,7JuxR74u5BSZpfIvNdRvFb,0.982000,0.150,0.1790,0.000019,9,0.112,-19.900,84.288,0,0.0463,0.174
902,1nhXX3flwQf9Rq4xL9ixJL,0.439000,0.449,0.5090,0.000021,9,0.347,-6.944,106.886,0,0.0310,0.164


In [ ]:
merged_df = track_data.merge(features_df, on="track_id", how="left")
merged_df

,artists,tracks,artist_id,track_id,track_popularity,track_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,tempo,mode,speechiness,valence
0,Alesana,curse of the virgin canvas,1fnDlUNi67FQtHHqBl4Yfg,6q2RsgScL1R3dG9pAbZzhy,43.0,None,0.009310,0.228,0.878,0.000114,2.0,0.750,-5.232,193.133,0.0,0.1290,0.161
1,Alesana,the artist,1fnDlUNi67FQtHHqBl4Yfg,1SCJh0eUMRLsm9xKEb6W2z,39.0,None,0.003200,0.264,0.979,0.000002,2.0,0.228,-3.378,174.119,0.0,0.2390,0.472
2,Alesana,"congratulations, i hate you",1fnDlUNi67FQtHHqBl4Yfg,2wiRaJG78MwRqBWnDSoO4H,40.0,https://p.scdn.co/mp3-preview/d657219662766b9b...,0.000058,0.318,0.919,0.011800,2.0,0.104,-5.928,133.762,0.0,0.0879,0.186
3,Alesana,a lunatic s lament,1fnDlUNi67FQtHHqBl4Yfg,3soK40VuGFJaqY5BeHG5c6,39.0,None,0.000794,0.315,0.916,0.001690,2.0,0.657,-4.647,186.084,0.0,0.0892,0.136
4,Alesana,ambrosia,1fnDlUNi67FQtHHqBl4Yfg,0CsngWR4MiYHiDbBebVxql,45.0,None,0.000038,0.275,0.984,0.482000,7.0,0.112,-4.328,119.376,0.0,0.0601,0.319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Anonymous,is getting rich worth it?,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Anonymous,the grave,4kCZ5nyurc9eIqLJfUcW0Y,7JuxR74u5BSZpfIvNdRvFb,0.0,https://p.scdn.co/mp3-preview/c053a3ed0ce07b7e...,0.982000,0.150,0.179,0.000019,9.0,0.112,-19.900,84.288,0.0,0.0463,0.174
997,Anonymous,alisoun,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Anouk,it s a new day,6ltVunYjAAD70YtVO6rxvX,1nhXX3flwQf9Rq4xL9ixJL,37.0,None,0.439000,0.449,0.509,0.000021,9.0,0.347,-6.944,106.886,0.0,0.0310,0.164


In [ ]:
# Downloading these songs according to track id
track_data.set_index('track_id',inplace=True)
track_data.to_csv('track_data.csv')

In [ ]:
track_data

In [ ]:
response = requests.get(query['tracks']['items'][0]['preview_url'], verify=False) # using track uris
audio_fn = os.path.join(audio_path,*track_id[:2],track_id+'.mp3') 
open(f"downloaded_songs/{artists_10[i]}/{top_10_track_names_clean[song]}.mp3", 'wb').write(response.content)  

In [ ]:
audio_path = '/Users/ytkd/Desktop/downloaded_songs'
# trac = "62bOmKYxYg7dhrC6gH9vFn"
# x = os.path.join(audio_path,trac[:1],trac+'.mp3')
# x

In [ ]:
os.mkdir('/Users/ytkd/Desktop/downloaded_songs')
audio_path = '/Users/ytkd/Desktop/downloaded_songs'
for i,url in enumerate(preview_url):
    if url is not None:
        response = requests.get(url, verify=False)
        if os.path.exists(f'{audio_path}/{track_id[i][:1]}') is False:
            os.mkdir(f'{audio_path}/{track_id[i][:1]}')
        open(f"{os.path.join(audio_path,track_id[i][:1],track_id[i]+'.mp3')}", 'wb').write(response.content)  




In [ ]:
count = 0
for id in preview_url:
    if id is None:
        count = count + 1
count
print(f'Out of 200 sample records the preview url is available for only {int(200-count)} songs')

In [ ]:
x = sp.search('artist: 5 Seconds of Summer track: she looks so perfect',type='track')
x['tracks']['items']

In [ ]:
test = sp.search('artist:Coldplay track:Fix You', type='track')
test['tracks']['href']

In [ ]:
import json
with open('test.json', 'w') as convert_file:
     convert_file.write(json.dumps(test))

In [ ]:
# create a histigram and check file names 